## Introduction ##
The idea here is to use a bag of visual words model to classify the different images. We will use SIFT algorithm to extract the keypoints of each image and create the bag of words.<br>
More information about this method can be found here:<br><ul>
<li>http://www.cs.cmu.edu/~16385/lectures/Lecture12.pdf</li>
<li>https://www.youtube.com/watch?v=iGZpJZhqEME</li>

Some part of this script are inside function, it's just a way to avoid error when I will publish this notebook. If you want to use this script, just remove line starting by "def ...".

In [11]:
import cv2
import numpy as np
import os
import pandas as pd
import csv

from sklearn.cluster import MiniBatchKMeans
from sklearn.neural_network import MLPClassifier

import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

import os
from PIL import Image
import cv2

In [12]:
#Setup Gdrive file download extention 
!conda install -y gdown

Channels:
 - rapidsai
 - nvidia
 - conda-forge
 - defaults
Platform: linux-64

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7bf506a6b9a0>: Failed to resolve 'conda.anaconda.org' ([Errno -3] Temporary failure in name resolution)")': /conda-forge/noarch/repodata.json.zst

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7bf506a9bbb0>: Failed to resolve 'conda.anaconda.org' ([Errno -3] Temporary failure in name resolution)")': /rapidsai/linux-64/repodata.json.zst

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7bf505cf47c0>: Failed to resolve 'conda.anaconda.org' ([Errno -3] Temporary failure in name resoluti

In [3]:
!gdown  1zbmXm-A-wAKoPuQ_fV1tfMi2KXsOKRw4
!gdown  1qxYTECYa6_uHSW4V61WzX54BgcZ9hCKe
!gdown  1fX_2IDL06OGvLQtnhvFzBy_BDsdaa2Dz

/bin/bash: gdown: command not found
/bin/bash: gdown: command not found
/bin/bash: gdown: command not found


In [4]:
#extract
!tar -xf /kaggle/working/annot.tar
!tar -xf /kaggle/working/train.tar
!tar -xf /kaggle/working/val.tar

tar: /kaggle/working/annot.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
tar: /kaggle/working/train.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
tar: /kaggle/working/val.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [5]:
train_dir = '/kaggle/working/train_set'
test_dir = '/kaggle/working/val_set'

In [6]:
df = pd.read_csv('/kaggle/working/train_info.csv', header=None)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/train_info.csv'

To do it, we will use OpenCV (cv2) library to extract keypoints with SIFT algorithm.

## Extract keypoints from each image ##

In [ ]:
img_path = '../input/images/'
train = pd.read_csv('../input/train.csv')
species = train.species.sort_values().unique()

dico = []

for leaf in train.id:
    img = cv2.imread(img_path + str(leaf) + ".jpg")
    kp, des = sift.detectAndCompute(img, None)

    for d in des:
        dico.append(d)

## Clustering  ##
We now have an array with a huge number of descriptors. We cannot use all of them to create or model so we need to cluster them. A rule-of-thumb is to create k centers with k = number of categories * 10 (in our case, it's 990).

In [ ]:
k = np.size(species) * 10

batch_size = np.size(os.listdir(img_path)) * 3
kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1).fit(dico)

I use MiniBatchKMeans to avoid Memory Error.

## Creation of the histograms ##
To create our each image by a histogram. We will create a vector of k value for each image. For each keypoints in an image, we will find the nearest center and increase by one its value.

In [ ]:
kmeans.verbose = False

histo_list = []

for leaf in train.id:
    img = cv2.imread(img_path + str(leaf) + ".jpg")
    kp, des = sift.detectAndCompute(img, None)

    histo = np.zeros(k)
    nkp = np.size(kp)

    for d in des:
        idx = kmeans.predict([d])
        histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

    histo_list.append(histo)

## Training of the neural network ##

In [ ]:
X = np.array(histo_list)
Y = []

# It's a way to convert species name into an integer
for s in train.species:
    Y.append(np.min(np.nonzero(species == s)))

mlp = MLPClassifier(verbose=True, max_iter=600000)
mlp.fit(X, Y)

## Predictions ##

In [ ]:
test = pd.read_csv('../input/test.csv')

result_file = open("sift.csv", "w")
result_file_obj = csv.writer(result_file)
result_file_obj.writerow(np.append("id", species))

for leaf in test.id:
    img = cv2.imread(img_path + str(leaf) + ".jpg")
    kp, des = sift.detectAndCompute(img, None)

    x = np.zeros(k)
    nkp = np.size(kp)

    for d in des:
        idx = kmeans.predict([d])
        x[idx] += 1/nkp

    res = mlp.predict_proba([x])
    row = []
    row.append(leaf)

    for e in res[0]:
        row.append(e)

    result_file_obj.writerow(row)

result_file.close()

## Alternative ##
I also run this script with ORB instead of SIFT and I got best results. To do it, just replace `cv2.xfeatures2d.SIFT_create()` by `cv2.ORB_create()`.